#### Import libraries

In [13]:
import pandas as pd
import matplotlib.pyplot as plt

### Read data from CSV Files - covid_large.csv and Hospital_beds.csv

In [19]:
# read data from covid_large csv file
covid_large_df = pd.read_csv("../../../../data/stage_3/covid_large.csv")
covid_large_df.tail()

,State,countyFIPS,County Name,Date,Num of Cases,Num of Deaths,population
1829584,WY,56045,weston county,2021-08-12,700,6,6927
1829585,WY,56045,weston county,2021-08-13,700,6,6927
1829586,WY,56045,weston county,2021-08-14,705,6,6927
1829587,WY,56045,weston county,2021-08-15,705,6,6927
1829588,WY,56045,weston county,2021-08-16,705,6,6927


In [20]:
# read data from Hospital_beds csv file
hospital_beds_df = pd.read_csv("../../../../data/stage_1/Definitive_Healthcare%3A_USA_Hospital_Beds.csv")
hospital_beds_df.head()

,X,Y,OBJECTID,HOSPITAL_NAME,HOSPITAL_TYPE,HQ_ADDRESS,HQ_ADDRESS1,HQ_CITY,HQ_STATE,HQ_ZIP_CODE,...,CNTY_FIPS,FIPS,NUM_LICENSED_BEDS,NUM_STAFFED_BEDS,NUM_ICU_BEDS,ADULT_ICU_BEDS,PEDI_ICU_BEDS,BED_UTILIZATION,Potential_Increase_In_Bed_Capac,AVG_VENTILATOR_USAGE
0,-81.510688,41.133034,3001,Western Reserve Hospital (FKA Summa Western Re...,Short Term Acute Care Hospital,1900 23rd St,NaN,Cuyahoga Falls,OH,44223.0,...,153.0,39153.0,272.0,83.0,12,12,0.0,0.430764,189,2.0
1,-81.500489,41.080189,3002,Summa Health System - Akron Campus,Short Term Acute Care Hospital,525 E Market St,NaN,Akron,OH,44304.0,...,153.0,39153.0,649.0,584.0,64,64,0.0,0.540098,65,30.0
2,-81.589456,41.014805,3003,Summa Health System - Barberton Campus,Short Term Acute Care Hospital,155 5th St Ne,NaN,Barberton,OH,44203.0,...,153.0,39153.0,271.0,158.0,12,12,0.0,0.628819,113,0.0
3,-81.532470,41.078325,3004,Akron General,Short Term Acute Care Hospital,1 Akron General Ave,NaN,Akron,OH,44307.0,...,153.0,39153.0,383.0,383.0,34,34,0.0,0.706661,0,10.0
4,-81.513552,41.096977,3005,Crystal Clinic Orthopaedic Center,Short Term Acute Care Hospital,444 N Main St,NaN,Akron,OH,44310.0,...,153.0,39153.0,94.0,59.0,10,10,0.0,0.431066,35,2.0


### Calculation of COVID-19 data trends for last week of the data for CALIFORNIA State

In [21]:
# Extracting California State Data from covid_large_df between 09/07/2020 to 09/14/2020
COVID_CA = covid_large_df.loc[(covid_large_df["countyFIPS"] != 0) & (covid_large_df["State"] == "CA") & (covid_large_df["Date"] >= "2020-09-07")]
COVID_CA.head()

,State,countyFIPS,County Name,Date,Num of Cases,Num of Deaths,population
109672,CA,6001,alameda county,2020-09-07,19350,300,1671329
109673,CA,6001,alameda county,2020-09-08,19350,300,1671329
109674,CA,6001,alameda county,2020-09-09,19596,303,1671329
109675,CA,6001,alameda county,2020-09-10,19710,303,1671329
109676,CA,6001,alameda county,2020-09-11,19819,320,1671329


### Merge of COVID-19 dataset with Hospital Beds Enrichment dataset 

#### Preprocessing of Hospital Beds Enrichment dataset

In [22]:
#Droppping few columns which might be repeatitive after merge
hospital_beds_df=hospital_beds_df.drop(['X','Y','HQ_ADDRESS1','HQ_STATE','COUNTY_NAME','STATE_FIPS','CNTY_FIPS'], axis=1)

#Dropping rows having NAN in FIPS column
hospital_beds_df = hospital_beds_df[hospital_beds_df['FIPS'].notna()]

#Converted all NAN cells to 0
hospital_beds_df = hospital_beds_df.fillna(0)

#Converted few float columns to int
hospital_beds_df = hospital_beds_df.astype({'FIPS': 'int','NUM_LICENSED_BEDS':'int','NUM_STAFFED_BEDS':'int','PEDI_ICU_BEDS':'int'})

hospital_beds_df.head()

,OBJECTID,HOSPITAL_NAME,HOSPITAL_TYPE,HQ_ADDRESS,HQ_CITY,HQ_ZIP_CODE,STATE_NAME,FIPS,NUM_LICENSED_BEDS,NUM_STAFFED_BEDS,NUM_ICU_BEDS,ADULT_ICU_BEDS,PEDI_ICU_BEDS,BED_UTILIZATION,Potential_Increase_In_Bed_Capac,AVG_VENTILATOR_USAGE
0,3001,Western Reserve Hospital (FKA Summa Western Re...,Short Term Acute Care Hospital,1900 23rd St,Cuyahoga Falls,44223.0,Ohio,39153,272,83,12,12,0,0.430764,189,2.0
1,3002,Summa Health System - Akron Campus,Short Term Acute Care Hospital,525 E Market St,Akron,44304.0,Ohio,39153,649,584,64,64,0,0.540098,65,30.0
2,3003,Summa Health System - Barberton Campus,Short Term Acute Care Hospital,155 5th St Ne,Barberton,44203.0,Ohio,39153,271,158,12,12,0,0.628819,113,0.0
3,3004,Akron General,Short Term Acute Care Hospital,1 Akron General Ave,Akron,44307.0,Ohio,39153,383,383,34,34,0,0.706661,0,10.0
4,3005,Crystal Clinic Orthopaedic Center,Short Term Acute Care Hospital,444 N Main St,Akron,44310.0,Ohio,39153,94,59,10,10,0,0.431066,35,2.0


#### Merge of covid_large_df and hospital_beds_df

In [23]:
#Merged covid_large_df and hospital_beds_df using inner join 
#by joining on "countyFIPS" in covid_large_df with "FIPS" in hospital_beds_df
COVID_HospitalBeds_merged_df = pd.merge(covid_large_df,hospital_beds_df,how="inner",left_on="countyFIPS",right_on="FIPS")
COVID_HospitalBeds_merged_df = COVID_HospitalBeds_merged_df.drop(['OBJECTID','HQ_ADDRESS','FIPS'], axis=1)
COVID_HospitalBeds_merged_df

,State,countyFIPS,County Name,Date,Num of Cases,Num of Deaths,population,HOSPITAL_NAME,HOSPITAL_TYPE,HQ_CITY,HQ_ZIP_CODE,STATE_NAME,NUM_LICENSED_BEDS,NUM_STAFFED_BEDS,NUM_ICU_BEDS,ADULT_ICU_BEDS,PEDI_ICU_BEDS,BED_UTILIZATION,Potential_Increase_In_Bed_Capac,AVG_VENTILATOR_USAGE
0,AL,1001,autauga county,2020-01-22,0,0,55869,Prattville Baptist Hospital,Short Term Acute Care Hospital,Prattville,36067.0,Alabama,85,55,6,6,0,0.623113,30,2.0
1,AL,1001,autauga county,2020-01-23,0,0,55869,Prattville Baptist Hospital,Short Term Acute Care Hospital,Prattville,36067.0,Alabama,85,55,6,6,0,0.623113,30,2.0
2,AL,1001,autauga county,2020-01-24,0,0,55869,Prattville Baptist Hospital,Short Term Acute Care Hospital,Prattville,36067.0,Alabama,85,55,6,6,0,0.623113,30,2.0
3,AL,1001,autauga county,2020-01-25,0,0,55869,Prattville Baptist Hospital,Short Term Acute Care Hospital,Prattville,36067.0,Alabama,85,55,6,6,0,0.623113,30,2.0
4,AL,1001,autauga county,2020-01-26,0,0,55869,Prattville Baptist Hospital,Short Term Acute Care Hospital,Prattville,36067.0,Alabama,85,55,6,6,0,0.623113,30,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749134,WY,56045,weston county,2021-08-12,700,6,6927,Weston County Health Services,Critical Access Hospital,Newcastle,82701.0,Wyoming,12,12,2,2,0,0.599641,0,0.0
3749135,WY,56045,weston county,2021-08-13,700,6,6927,Weston County Health Services,Critical Access Hospital,Newcastle,82701.0,Wyoming,12,12,2,2,0,0.599641,0,0.0
3749136,WY,56045,weston county,2021-08-14,705,6,6927,Weston County Health Services,Critical Access Hospital,Newcastle,82701.0,Wyoming,12,12,2,2,0,0.599641,0,0.0
3749137,WY,56045,weston county,2021-08-15,705,6,6927,Weston County Health Services,Critical Access Hospital,Newcastle,82701.0,Wyoming,12,12,2,2,0,0.599641,0,0.0


#### Exporting COVID_HospitalBeds_merged_df to path '../data/COVID_HospitalBeds_merged_df.csv'

In [24]:
COVID_HospitalBeds_merged_df.to_csv('../../../../data/stage_3/COVID_HospitalBeds_merged_df.csv',index=False)

#### Reading from Exported Location

In [25]:
pd.read_csv('../../../../data/stage_3/COVID_HospitalBeds_merged_df.csv')

,State,countyFIPS,County Name,Date,Num of Cases,Num of Deaths,population,HOSPITAL_NAME,HOSPITAL_TYPE,HQ_CITY,HQ_ZIP_CODE,STATE_NAME,NUM_LICENSED_BEDS,NUM_STAFFED_BEDS,NUM_ICU_BEDS,ADULT_ICU_BEDS,PEDI_ICU_BEDS,BED_UTILIZATION,Potential_Increase_In_Bed_Capac,AVG_VENTILATOR_USAGE
0,AL,1001,autauga county,2020-01-22,0,0,55869,Prattville Baptist Hospital,Short Term Acute Care Hospital,Prattville,36067.0,Alabama,85,55,6,6,0,0.623113,30,2.0
1,AL,1001,autauga county,2020-01-23,0,0,55869,Prattville Baptist Hospital,Short Term Acute Care Hospital,Prattville,36067.0,Alabama,85,55,6,6,0,0.623113,30,2.0
2,AL,1001,autauga county,2020-01-24,0,0,55869,Prattville Baptist Hospital,Short Term Acute Care Hospital,Prattville,36067.0,Alabama,85,55,6,6,0,0.623113,30,2.0
3,AL,1001,autauga county,2020-01-25,0,0,55869,Prattville Baptist Hospital,Short Term Acute Care Hospital,Prattville,36067.0,Alabama,85,55,6,6,0,0.623113,30,2.0
4,AL,1001,autauga county,2020-01-26,0,0,55869,Prattville Baptist Hospital,Short Term Acute Care Hospital,Prattville,36067.0,Alabama,85,55,6,6,0,0.623113,30,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749134,WY,56045,weston county,2021-08-12,700,6,6927,Weston County Health Services,Critical Access Hospital,Newcastle,82701.0,Wyoming,12,12,2,2,0,0.599641,0,0.0
3749135,WY,56045,weston county,2021-08-13,700,6,6927,Weston County Health Services,Critical Access Hospital,Newcastle,82701.0,Wyoming,12,12,2,2,0,0.599641,0,0.0
3749136,WY,56045,weston county,2021-08-14,705,6,6927,Weston County Health Services,Critical Access Hospital,Newcastle,82701.0,Wyoming,12,12,2,2,0,0.599641,0,0.0
3749137,WY,56045,weston county,2021-08-15,705,6,6927,Weston County Health Services,Critical Access Hospital,Newcastle,82701.0,Wyoming,12,12,2,2,0,0.599641,0,0.0


#### Exporting sample rows from COVID_HospitalBeds_merged_df to upload the csv to GIT

In [26]:
COVID_HospitalBeds_merged_df.head(500000).to_csv('../../../../data/stage_3/COVID_HospitalBeds_merged_sample_df.csv',index=False)